In [ ]:
!nvidia-smi

In [ ]:
pip install transformers datasets evaluate seqeval[cuda]


In [ ]:
pip install pandas

In [ ]:
pip install mpi4py

In [ ]:
pip install transformers seqeval[gpu]

In [ ]:
pip install scikit-learn

In [ ]:
pip install ray

In [ ]:
pip install pydantic==1.10.13

In [ ]:
pip install modin

https://data.world/collery/post-office-data/workspace/file?filename=USPO+Change+of+Address+data+Y2022.csv
https://data.world/len/us-first-names-database

https://www.kaggle.com/datasets/ahmedshahriarsakib/list-of-real-usa-addresses
https://www.kaggle.com/datasets/datagov/usa-names

https://data.world/adamhelsinger/directory-of-representatives
https://data.world/state-of-connecticut/ikyf-f2iu

In [1]:
import pandas as pd
# import modin.pandas as pd
import ray

In [ ]:
import time

ray.init()

In [2]:
# import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
# Read the CSV file
surnames = pd.read_csv("data/Common_Surnames_Census_2000.csv")

# View the first 5 rows
surnames.head()

,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56
1,JOHNSON,2,1857160,688.44,1569.30,61.55,33.80,0.42,0.91,1.82,1.50
2,WILLIAMS,3,1534042,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60
3,BROWN,4,1380145,511.62,2649.58,60.71,34.54,0.41,0.83,1.86,1.64
4,JONES,5,1362755,505.17,3154.75,57.69,37.73,0.35,0.94,1.85,1.44


In [ ]:
surnames = surnames.drop(['rank', 'count', 'prop100k', 'cum_prop100k', 'pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace', 'pcthispanic'], axis=1)

In [ ]:
surnames.shape

In [ ]:
surnames.head()

In [ ]:
surnames = surnames.drop_duplicates(subset=['name'])

In [ ]:
surnames = surnames.rename(columns={"name": "lastname"})

In [ ]:
# Read the CSV file
firstnameandgender = pd.read_csv("data/SSA_Names_DB.csv")

# View the first 5 rows
firstnameandgender.head()

In [ ]:
firstnameandgender = firstnameandgender.drop(['Frequency', 'Include?',], axis=1)

In [ ]:
firstnameandgender.shape

In [ ]:
firstnameandgender = firstnameandgender.drop_duplicates(subset=['Name'])

In [ ]:
firstnameandgender.shape

In [ ]:
firstnameandgender = firstnameandgender.rename(columns={"Name": "firstname", "Gender": "gender"})

In [ ]:
firstnameandgender.shape

In [ ]:
firstnameandgender.head()

In [ ]:
# Read the CSV file
address = pd.read_csv("data/USPO Change of Address data Y2022.csv")

# View the first 5 rows
address.head()

In [ ]:
address = address.drop(['YYYYMM', 'TOTAL FROM ZIP',"TOTAL BUSINESS", "TOTAL FAMILY", "TOTAL INDIVIDUAL", "TOTAL PERM", "TOTAL TEMP"], axis=1)

In [ ]:
address = address.drop(['TOTAL TO ZIP', 'TOTAL BUSINESS.1',"TOTAL FAMILY.1", "TOTAL INDIVIDUAL.1", "TOTAL PERM.1", "TOTAL TEMP.1"], axis=1)

In [ ]:
address.head()

In [ ]:
address['ZIPCODE'] = address['ZIPCODE'].apply(lambda x: x.replace('=', '').replace('"', ''))


In [ ]:
address.head()

In [ ]:
address = address.rename(columns={"ZIPCODE": "zipcode", "CITY": "city", "STATE": "state"})

In [ ]:
address.shape

In [ ]:
lst=address.city.unique() 

In [ ]:
len(lst)

In [ ]:
# shuffle the data
address = address.sample(frac = 1)
firstnameandgender = firstnameandgender.sample(frac = 1)
surnames = surnames.sample(frac = 1)

In [ ]:
address.reset_index(inplace = True,  drop=True)
firstnameandgender.reset_index(inplace = True,  drop=True)
surnames.reset_index(inplace = True,  drop=True)


In [ ]:
surnames=surnames.iloc[:30460]

In [ ]:
firstnameandgender=firstnameandgender.iloc[:30460]

In [ ]:
address=address.iloc[:30460]

In [ ]:
address.shape

In [ ]:
firstnameandgender['firstname'] = firstnameandgender['firstname'].str.lower()

In [ ]:
firstnameandgender['gender'] = firstnameandgender['gender'].str.lower()

In [ ]:
surnames['lastname'] = surnames['lastname'].str.lower()

In [ ]:
firstnameandgender.head()

In [ ]:
from uuid import uuid4
firstnameandgender['uuid'] = firstnameandgender.index.to_series().map(lambda x: uuid4())


In [ ]:
surnames['uuid'] = surnames.index.to_series().map(lambda x: uuid4())

In [ ]:
address['uuid'] = address.index.to_series().map(lambda x: uuid4())

In [ ]:
firstnameandgender.loc[:,['uuid','firstname','gender']]


In [ ]:
surnames.loc[:,['uuid','lastname']]

In [ ]:
address.loc[:,['uuid','city', 'state','zipcode']]

In [ ]:
address['city'] = address['city'].str.lower()
address['state'] = address['state'].str.lower()

In [ ]:
horizontal_concat = pd.concat([firstnameandgender, surnames, address], axis=1)


In [ ]:
horizontal_concat.head()

In [ ]:
horizontal_concat.reset_index(inplace=True, drop=True) 


In [ ]:
horizontal_concat.shape

In [ ]:
horizontal_concat.to_csv("./data/horizontal_data.csv", index=False)

In [ ]:
horizontal_concat['firstname'] = horizontal_concat['firstname'].str.lower()

In [ ]:
horizontal_concat['gender'] = horizontal_concat['gender'].str.lower()

In [ ]:
horizontal_concat['lastname'] = horizontal_concat['lastname'].str.lower()

In [ ]:
horizontal_concat['city'] = horizontal_concat['city'].str.lower()

In [ ]:
horizontal_concat['state'] = horizontal_concat['state'].str.lower()

In [ ]:
horizontal_concat

In [ ]:
#Feature Engineering
# Melt the DataFrame to convert columns to rows
melted_data = pd.melt(horizontal_concat, var_name='Entity', value_name='Value')
 
# 'Entity' column now contains the labels for the entities (former column names)
# 'Value' column contains the corresponding values
 
print(melted_data)

In [ ]:
labels=[
                   "O",
                    "firstname",
                    "lastname",
                    "city",
                    "state",
                    "zipcode",
                ]

In [ ]:
# Define the labels for each column
labels = ['firstname', 'gender', 'lastname', 'zipcode', 'city', 'state']


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset


In [ ]:
label_list = wnut["train"].features[f"ner_tags"].feature.names


In [ ]:
label_list

In [ ]:
wnut["train"]

In [ ]:
labels = ['firstname', 'lastname', 'zipcode', 'city', 'state']


In [ ]:
len(firstnameandgender)

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:

firstnametrain, firstnametest = train_test_split(firstnameandgender.loc[:,['uuid','firstname']], test_size=0.2)


surnametrain, surnametest = train_test_split(surnames.loc[:,['uuid','lastname']], test_size=0.2)


addresscitytrain, addresscitytest = train_test_split(address.loc[:,['uuid','city']], test_size=0.2)

addressstatetrain, addressstatetest = train_test_split(address.loc[:,['uuid','state']], test_size=0.2)

addressziptrain, addressziptest = train_test_split(address.loc[:,['uuid','zipcode']], test_size=0.2)

In [ ]:
len(firstnametrain)

In [ ]:
firstnametrain.head()

In [ ]:
column_names=["id","token", "ner_tag"]
dat = []
traindata=pd.DataFrame(dat, columns=column_names)
for index, row in firstnametrain.iterrows():
    traindata.loc[len(traindata)]=[ str(row['uuid']),row['firstname'], "firstname" ]


In [ ]:
for index, row in surnametrain.iterrows():
    traindata.loc[len(traindata)]=[ str(row['uuid']),row['lastname'], "lastname"]


In [ ]:
for index, row in addresscitytrain.iterrows():
    traindata.loc[len(traindata)]=[ str(row['uuid']),row['city'], "city"]

In [ ]:

for index, row in addressstatetrain.iterrows():
    traindata.loc[len(traindata)]=[ str(row['uuid']),row['state'], "state"]


In [ ]:
traindata.columns

In [ ]:
for index, row in addressziptrain.iterrows():
    traindata.loc[len(traindata)]=[ str(row['uuid']),row['zipcode'], "zipcode"]

In [ ]:
test_column_names=["id","token", "ner_tag"]
testdat = []
testdata=pd.DataFrame(testdat, columns=test_column_names)

In [ ]:
for index, row in firstnametest.iterrows():
    testdata.loc[len(testdata)]=[str(row['uuid']),row['firstname'], "firstname"]

In [ ]:
for index, row in surnametest.iterrows():
    testdata.loc[len(testdata)]=[ str(row['uuid']),row['lastname'], "lastname"]

In [ ]:
for index, row in addresscitytest.iterrows():
    testdata.loc[len(testdata)]=[ str(row['uuid']),row['city'], "city"]

In [ ]:
for index, row in addressstatetest.iterrows():
    testdata.loc[len(testdata)]=[ str(row['uuid']),row['state'], "state"]

In [ ]:

for index, row in addressziptest.iterrows():
    testdata.loc[len(testdata)]=[ str(row['uuid']),row['zipcode'], "zipcode"]

In [ ]:
testdata.columns

In [ ]:
testdata.head()


In [ ]:
from datasets import Dataset, Value, ClassLabel, Features
import pandas as pd
features = Features({"id": Value("string"), "token": Value("string"), "ner_tag": ClassLabel(num_classes=5, names=["firstname","lastname","city","state","zipcode"])})

train_dataset = Dataset.from_dict(traindata, features)
test_dataset = Dataset.from_dict(testdata, features)

In [ ]:
from datasets import DatasetDict

In [ ]:
train_dataset

In [ ]:
test_dataset

In [ ]:
train_dataset.save_to_disk("./data/traindatasethg_name")

In [ ]:
test_dataset.save_to_disk("./data/testdatasethg_name")

In [5]:
from datasets import concatenate_datasets, Dataset, load_from_disk, DatasetDict

train_dataset=load_from_disk("./data/traindatasethg_name")
test_dataset=load_from_disk("./data/testdatasethg_name")

In [6]:
train_dataset[len(train_dataset)-100]

{'id': '78cd9b71-c67f-44d9-acc5-e64c36a8b510', 'token': '08403', 'ner_tag': 4}

In [7]:
finalds = DatasetDict({
    'train': train_dataset,
    'test': test_dataset})

In [8]:
label_list = finalds["train"].features[f"ner_tag"].names


In [9]:
label_list

['firstname', 'lastname', 'city', 'state', 'zipcode']

In [10]:
labels=label_list
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
label2id

{'firstname': 0, 'lastname': 1, 'city': 2, 'state': 3, 'zipcode': 4}

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
example = finalds["train"][0]
tokenized_input = tokenizer(example["token"], is_split_into_words=False)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]', 'maud', '##ie', '[SEP]']

In [13]:
 finalds["test"][0]

{'id': '25c1f7b9-f808-488e-86d6-3edd52aa876a', 'token': 'calah', 'ner_tag': 0}

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["token"], truncation=True, is_split_into_words=False)
    tokenized_inputs["labels"] = examples[f"ner_tag"]
    return tokenized_inputs

In [15]:
tokenized_input = finalds.map(tokenize_and_align_labels, batched=True)


In [16]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

2023-12-04 14:13:19.430112: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 14:13:19.478408: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 14:13:19.478441: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 14:13:19.478469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 14:13:19.487822: I tensorflow/core/platform/cpu_feature_g

In [17]:
import evaluate

seqeval = evaluate.load("seqeval")

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        # print(prediction, label, tokens)
        for (p, l, t) in zip(prediction, label, tokens):
            # print(l, p, tokenizer.convert_ids_to_tokens(int(t)))
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith(PRE_WORD):
                # print('append')
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])
    # print(true_predictions, true_labels)

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


def compute_metrics(p):
    preds, labels = p

    preds = np.argmax(preds, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    
    precission = precission_score(labels, preds, average='weighted')
    recall=recall_score(labels, preds, average="weighted")
    f1=f1_score(labels, preds, average='weighted')
    
    
    return {
        'accuracy':  accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        # [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        # [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=predictions, references=labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        # print(prediction, label, tokens)
        for (p, l, t) in zip(prediction, label, tokens):
            # print(l, p, tokenizer.convert_ids_to_tokens(int(t)))
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith(PRE_WORD):
                # print('append')
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])
    # print(true_predictions, true_labels)

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [18]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_input["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

2023-12-04 14:13:35.183880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1060 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [19]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=5, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [20]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_input["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_input["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
import tensorflow as tf

model.compile(optimizer=optimizer, metrics=[compute_metrics]) 

In [22]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

Epoch 1/3


2023-12-04 14:13:54.496505: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1ee44d3190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-04 14:13:54.496537: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9
2023-12-04 14:13:54.502952: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-04 14:13:54.525239: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8904
2023-12-04 14:13:54.625148: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7615/7615 [==============================] - 271s 33ms/step - loss: 0.2987 - val_loss: 0.2518
Epoch 2/3
7615/7615 [==============================] - 247s 32ms/step - loss: 0.2060 - val_loss: 0.2431
Epoch 3/3
7615/7615 [==============================] - 247s 32ms/step - loss: 0.1524 - val_loss: 0.2525


In [23]:
result = model.evaluate(x=tf_validation_set)

1904/1904 [==============================] - 18s 9ms/step - loss: 0.2525


In [ ]:
presult = model.predict(x=tf_validation_set)

In [ ]:
presult

In [ ]:
tokenized_input

In [ ]:
from sklearn.metrics import confusion_matrix
y_test=tokenized_input['test']["ner_tag"]
y_pred = np.argmax(presult, axis=0)
conf_mat = confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
y_test=tokenized_inputs["labels"]
y_pred = model.predict(x=tf_validation_set)
# y_pred = np.argmax(y_pred, axis=2)

conf_mat = confusion_matrix(y_test, y_pred)

In [ ]:
model.save_pretrained("./model/classification-ner")
tokenizer.save_pretrained("./model/classification-ner")

In [35]:
from transformers import TextClassificationPipeline

text = ["sihag"]
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)
pipe(text)

[{'label': 'firstname', 'score': 0.8217609524726868}]

In [ ]:

Txt: File
